In [0]:
# !kill -9 -1

In [1]:
!pip install keras --upgrade

    100% |████████████████████████████████| 317kB 13.9MB/s 
  Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp /content/gdrive/My\ Drive/speed/* .

In [4]:
ls

gdrive/  images.zip  sample_data/  train.txt  weights.h5


In [0]:
!unzip -q images.zip

In [6]:
ls

gdrive/  images/  images.zip  sample_data/  train.txt  weights.h5


In [7]:
import glob

X = sorted(glob.glob('images/*'))
print(len(X))

20400


In [8]:
f = open('train.txt', 'r')
y = f.read().split('\n')
f.close()

print(len(y))

20400


In [9]:
import pandas as pd
from sklearn.utils import shuffle

data_train, data_valid, data_test = [], [], []

for i in range(len(X)):
    if i%4 in [0, 3]:
        data_train.append([X[i].split('/')[-1], float(y[i])])
    elif i%4 == 1:
        data_valid.append([X[i].split('/')[-1], float(y[i])])       
    elif i%4 == 2: 
        data_test.append([X[i].split('/')[-1], float(y[i])])
        
print(len(data_train), len(data_valid), len(data_test))
print(data_train[0], data_valid[0], data_test[0]) 
        
data_train = shuffle(data_train) 
data_valid = shuffle(data_valid)
data_test = shuffle(data_test) 
        
print(data_train[0], data_valid[0], data_test[0])        

10200 5100 5100
['000000.jpg', 28.105569] ['000001.jpg', 28.105569] ['000002.jpg', 28.106527]
['003248.jpg', 21.573246] ['009441.jpg', 3.901046] ['010826.jpg', 6.366494]


In [10]:
train_frame = pd.DataFrame(data_train, columns=['file_name', 'value'])
valid_frame = pd.DataFrame(data_valid, columns=['file_name', 'value'])
test_frame = pd.DataFrame(data_test, columns=['file_name', 'value'])
print(test_frame)

       file_name      value
0     010826.jpg   6.366494
1     000322.jpg  18.120178
2     016586.jpg   2.792415
3     011722.jpg   7.265192
4     003574.jpg  21.914365
5     006538.jpg  15.499196
6     013894.jpg   9.496358
7     006354.jpg  13.018755
8     010658.jpg   0.559678
9     009986.jpg   9.196102
10    017314.jpg  10.428071
11    016558.jpg   4.039448
12    017642.jpg  13.775368
13    017034.jpg   1.645719
14    014498.jpg   0.306252
15    008002.jpg  22.806586
16    018974.jpg   5.581401
17    005850.jpg  18.240682
18    004758.jpg  22.037887
19    010618.jpg   1.461054
20    016202.jpg  11.298519
21    015522.jpg   1.422830
22    000546.jpg  16.922980
23    009270.jpg   6.950560
24    016146.jpg   2.382183
25    013798.jpg  10.645881
26    005410.jpg  24.389572
27    008426.jpg  20.993471
28    011946.jpg   7.464756
29    018922.jpg   2.696704
...          ...        ...
5070  008134.jpg  25.631462
5071  006502.jpg  14.520460
5072  010414.jpg   7.368636
5073  007770.jpg  24

In [11]:
import keras
import tensorflow as tf

print(keras.__version__)
print(tf.__version__)

Using TensorFlow backend.


2.2.4
1.12.0-rc2


In [0]:
from keras.applications import VGG16
from keras import models
from keras import layers
from keras import optimizers
from keras import callbacks
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator

def get_conv_base():
    
    conv_base = VGG16(weights='imagenet',
                      include_top=False,
                     input_shape=(150,150,3))

    conv_base.trainable = True

    set_trainable = False
    for layer in conv_base.layers:
        if layer.name == 'block5_conv1':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

    #conv_base.summary()
    
    return conv_base

def get_top_model():
    
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1))
    
    return model

In [0]:
def get_model():
    
    model = models.Sequential()
    model.add(get_conv_base())
    model.add(layers.Flatten())
    model.add(get_top_model())
    model.summary()
    
    return model

In [0]:
batch_size = 256
image_size = 150
lr = 0.001
num_epochs = 50

In [15]:
datagen = ImageDataGenerator(
        rescale=1./255)

train_generator = datagen.flow_from_dataframe(train_frame, './images', x_col='file_name',
                                              y_col='value', target_size=(image_size, image_size),
                                              batch_size=batch_size, class_mode='other')
valid_generator = datagen.flow_from_dataframe(valid_frame, './images', x_col='file_name',
                                              y_col='value', target_size=(image_size, image_size),
                                              batch_size=batch_size, class_mode='other')
test_generator = datagen.flow_from_dataframe(test_frame, './images', x_col='file_name',
                                              y_col='value', target_size=(image_size, image_size),
                                              batch_size=batch_size, class_mode='other')

Found 10200 images.
Found 5100 images.
Found 5100 images.


In [0]:
model = get_model()

model.compile(optimizer=optimizers.Adam(lr), loss='mse', metrics=['mae'])

checkpoint = callbacks.ModelCheckpoint('weights.h5',
                                        monitor='val_mean_absolute_error',
                                        mode='min', save_weights_only=True,
                                        save_best_only=True, verbose=1)

58892288/58889256 [==============================] - 2s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 2130433   
Total params: 16,845,121
Trainable params: 9,209,857
Non-trainable params: 7,635,264
_________________________________________________________________


In [0]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_generator.samples//batch_size,
                              epochs=num_epochs, callbacks=[checkpoint],
                              validation_data=valid_generator,
                              validation_steps=valid_generator.samples//batch_size)

Epoch 1/50
39/39 [==============================] - 108s 3s/step - loss: 128.0137 - mean_absolute_error: 8.3476 - val_loss: 22.7005 - val_mean_absolute_error: 3.5013

Epoch 00001: val_mean_absolute_error improved from inf to 3.50130, saving model to weights.h5
Epoch 2/50
39/39 [==============================] - 98s 3s/step - loss: 22.0833 - mean_absolute_error: 3.5472 - val_loss: 9.5540 - val_mean_absolute_error: 2.3632

Epoch 00002: val_mean_absolute_error improved from 3.50130 to 2.36323, saving model to weights.h5
Epoch 3/50
39/39 [==============================] - 85s 2s/step - loss: 15.7198 - mean_absolute_error: 2.9554 - val_loss: 7.6214 - val_mean_absolute_error: 2.1621

Epoch 00003: val_mean_absolute_error improved from 2.36323 to 2.16209, saving model to weights.h5
Epoch 4/50
39/39 [==============================] - 88s 2s/step - loss: 12.0805 - mean_absolute_error: 2.5408 - val_loss: 4.1635 - val_mean_absolute_error: 1.5910

Epoch 00004: val_mean_absolute_error improved from 

In [0]:
def mse(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

In [17]:
model = get_model()
model.load_weights('weights.h5')
model.compile(optimizer=optimizers.Adam(lr), loss='mse', metrics=['mae', mse])

58892288/58889256 [==============================] - 3s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 2130433   
Total params: 16,845,121
Trainable params: 9,209,857
Non-trainable params: 7,635,264
_________________________________________________________________


In [22]:
evaluate = model.evaluate_generator(test_generator,
                                   steps=test_generator.samples//batch_size)

print('Test \n loss:{} \n mae:{} \n mse:{}'.format(evaluate[0], evaluate[1], evaluate[2]))

Test 
 loss:0.20567535177657478 
 mae:0.3492529564782193 
 mse:0.20567535177657478


In [0]:
from google.colab import files
files.download('weights.h5')